In [23]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")
cppinclude("ogdf/basic/graph_generators/deterministic.h")
cppinclude("ogdf/energybased/FMMMLayout.h")


G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.gridGraph(G, 10, 10,True, True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = "N%s" % n.index()
    GA.x[n] = (n.index() %10) * 50 
    GA.y[n] = (n.index()//10) * 50
    
for adj in G.nodes[0].adjEntries:
    print(adj.twinNode().index())
    if adj.twinNode().index() == 9:
        e = adj.theEdge()
        GA.bends[e].pushBack(ogdf.DPoint(-100,-100)) 
    
#SL = ogdf.SugiyamaLayout()
#SL=ogdf.FMMMLayout();
 
#SL.useHighLevelOptions(True);
#SL.unitEdgeLength(15.0);
#SL.newInitialPlacement(True);
#SL.qualityVersusSpeed(ogdf.FMMMOptions.QualityVsSpeed.GorgeousAndEfficient);
#SL.call(GA)
ogdf.GraphIO.write(GA, "sugiyama-simple.graphml")
og
GA

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug
1
9
10
90


In [4]:
help(ogdf.Graph)

Help on class Graph in module ogdf:

class Graph(cppyy.gbl.CPPInstance)
 |  cppyy object proxy (internal)
 |  
 |  Method resolution order:
 |      Graph
 |      cppyy.gbl.CPPInstance
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __assign__(...)
 |      ogdf::Graph& ogdf::Graph::operator=(const ogdf::Graph& G)
 |  
 |  __init__(...)
 |      Graph::Graph()
 |      Graph::Graph(const ogdf::Graph& G)
 |  
 |  adjEntryArrayTableSize(...)
 |      int ogdf::Graph::adjEntryArrayTableSize()
 |  
 |  allEdges = cppyy template proxy (internal)
 |  allNodes = cppyy template proxy (internal)
 |  chooseEdge(...)
 |      ogdf::EdgeElement* ogdf::Graph::chooseEdge(std::function<bool(ogdf::EdgeElement*)> includeEdge = [] (ogdf::edge)     {
 |              return true;
 |          }
 |      , bool isFastTest = true)
 |  
 |  chooseNode(...)
 |      ogdf::NodeElement* ogdf::Graph::chooseNode(std::function<bool(ogdf::NodeElement*)> includeNode = [] (ogdf::node)     {
 |              ret

In [6]:
G.genus()

0